In [1]:
import numpy as np
import matplotlib.pyplot as plt
import struct,os
from array import array as pyarray
from numpy import append, array, int8, uint8, zeros

def load_mnist(image_file, label_file):
    digits=np.arange(10)

    fname_image = os.path.join(image_file)
    fname_label = os.path.join(label_file)

    flbl = open(fname_label, 'rb')
    magic_nr, size = struct.unpack(">II", flbl.read(8))
    lbl = pyarray("b", flbl.read())
    flbl.close()

    fimg = open(fname_image, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", fimg.read(16))
    img = pyarray("B", fimg.read())
    fimg.close()

    ind = [ k for k in range(size) if lbl[k] in digits ]
    N = len(ind)

    images = zeros((N, rows*cols), dtype=uint8)
    labels = zeros((N, 1), dtype=int8)
    for i in range(len(ind)):
        images[i] = array(img[ ind[i]*rows*cols : (ind[i]+1)*rows*cols ]).reshape((1, rows*cols))
        labels[i] = lbl[ind[i]]

    return images, labels


train_image, train_label = load_mnist("train-images.idx3-ubyte", "train-labels.idx1-ubyte")
test_image, test_label = load_mnist("t10k-images.idx3-ubyte", "t10k-labels.idx1-ubyte")

$$
h(x)=\argmax\limits_{c \in y}P(c)\prod\limits_{i=1}^{d}P(x_i|c)
$$
$$
P(c)=\frac{|D_c|}{|D|}
$$
$$
P(x_i|c)=\frac{|D_{c,x_i}|}{|D_c|}
$$

In code:

len(D[c]) for $|D_c|$ 

pc[c] for $P(c)$




In [2]:
label_type=10
D=[[] for _ in range(label_type)]
for i in range(train_image.shape[0]):
    D[int(train_label[i])].append(train_image[i])
pc=[len(D[i])/train_label.shape[0] for i in range(label_type)]
print(pc)

[0.09871666666666666, 0.11236666666666667, 0.0993, 0.10218333333333333, 0.09736666666666667, 0.09035, 0.09863333333333334, 0.10441666666666667, 0.09751666666666667, 0.09915]


$D_{c,x_i}$ 表示 $D_c$ 中在第i个属性取值为$x_i$的样本组成的集合

xc[c][i][x[i]] for $|D_{c,x_i}|$

xc.shape (10,28*28,256)

In [3]:
xc=[[[0 for _ in range(256)] for _ in range(train_image.shape[1])] for _ in range(label_type)]

In [4]:
for c in range(label_type):
    for x in D[c]:
        for i in range(x.shape[0]):
            xc[c][i][x[i]]+=1 


Laplacian correction

对于未出现的属性

$$
\hat{P}(c)=\frac{|D_c|+1}{|D|+N}
$$

$$
\hat{P}(x_i|c)=\frac{|D_{c,x_i}|+1}{|D_c|+N_i}
$$

其中$N$为训练集D中可能出现的类别,$N_i$表示第$i$个属性可能的取值数。

（未出现时，$|D_c|$,$|D_{c,x_i}|$显然为0）

In [12]:
pred=[]
for i in range(test_image.shape[0]):
    rt=np.array([np.log(pc[j]) for j in range(10)])
    x=test_image[i]
    for j in range(x.shape[0]):
        for c in range(label_type):
            rt[c]+=np.log(xc[c][j][x[j]]/len(D[c]) if xc[c][j][x[j]]>0 else (xc[c][j][x[j]]+1)/(len(D[c])+256))
    pred.append(np.argmax(rt))
    # print(pred[len(pred)-1],test_label[i])

In [13]:
err=0
for i in range(test_image.shape[0]):
    if(pred[i]!=test_label[i]):
        err+=1
print("accuracy is")
print(1-err/test_label.shape[0])

accuracy is
0.8401000000000001
